In [1]:
!pip install numpy pandas
!pip install opencv-python
!pip install tensorflow scikit-learn matplotlib



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/eb/f1/828bbccc84a72db960a7d116f55f3f6aec9f5658f5d32ce9db20142d5742/tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/62/27/585859e72e117fe861c2079bcba35591a84f801e21bc1ab85bce6ce60305/scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/9f/6e/264673e64001b99d747aff5a288eca82826c024437a3694e19aed1decf46/matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for tensorflow-intel==2.18.0 from https://files.pythonhosted.org/packages/ae/4e/44ce609


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(r'C:\Users\user\Downloads\ISL_CSLRT_Corpus'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\user\Downloads\ISL_CSLRT_Corpus\ISL_CSLRT.txt
C:\Users\user\Downloads\ISL_CSLRT_Corpus\corpus_csv_files\ISL Corpus sign glosses.csv
C:\Users\user\Downloads\ISL_CSLRT_Corpus\corpus_csv_files\ISL_CSLRT_Corpus details.xlsx
C:\Users\user\Downloads\ISL_CSLRT_Corpus\corpus_csv_files\ISL_CSLRT_Corpus_frame_details.xlsx
C:\Users\user\Downloads\ISL_CSLRT_Corpus\corpus_csv_files\ISL_CSLRT_Corpus_word_details.xlsx
C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Sentence_Level\are you free today\1\are you free today 01.jpg
C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Sentence_Level\are you free today\1\are you free today 02.jpg
C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Sentence_Level\are you free today\1\are you free today 03.jpg
C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Sentence_Level\are you free today\1\are you free today 04.jpg
C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Sentence_Level\are you free today\1\are you free today 05.jpg
C:\Users\user\Downloads\ISL_CSLRT_

In [3]:
# Import necessary libraries
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, TimeDistributed, LSTM, Dense, GlobalAveragePooling2D, Dropout, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224       # Size expected by MobileNetV2
NUM_FRAMES = 10                        # Fixed number of frames per sample (pad or truncate)
BATCH_SIZE = 16
INITIAL_EPOCHS = 50                    # First phase training epochs with frozen CNN
FINETUNE_EPOCHS = 50                   # Additional epochs after unfreezing part of the CNN
TOTAL_EPOCHS = INITIAL_EPOCHS + FINETUNE_EPOCHS
DATASET_DIR = r'C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level'  # Change this path to your dataset path

In [6]:
def load_frame(image_path):
    """
    Loads an image from disk, resizes it, and applies MobileNetV2 preprocessing.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Error reading image: {image_path}")
    # Convert from BGR (OpenCV default) to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img = preprocess_input(img)  # This scales pixels as required by MobileNetV2
    return img

def load_dataset(dataset_dir):
    """
    Loads the dataset from the specified directory.
    
    Expected structure:
    dataset_dir/
         Word1/
             image1.jpg
             image2.jpg
             ... (possibly many images representing multiple samples)
         Word2/
             image1.jpg
             image2.jpg
         ...
         
    The function groups images into sequences of NUM_FRAMES. If there are more images,
    they are split into consecutive chunks. If there are fewer than NUM_FRAMES, the chunk is padded.
    
    Returns:
      X: numpy array of shape (num_samples, NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, 3)
      y: numpy array of integer labels for each sample
      label_map: dictionary mapping word names to integer indices
    """
    X, y = [], []
    label_map = {}
    current_label = 0

    # Loop through each word folder (each word is a label)
    for word in sorted(os.listdir(dataset_dir)):
        word_folder = os.path.join(dataset_dir, word)
        if os.path.isdir(word_folder):
            print(f"\nProcessing label: {word}")
            label_map[word] = current_label

            # List image files directly in the word folder (ignore subfolders)
            image_files = sorted([os.path.join(word_folder, f)
                                  for f in os.listdir(word_folder)
                                  if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            if not image_files:
                print(f"  No image files found in {word_folder}.")
                continue

            num_images = len(image_files)
            # Group the images into consecutive sequences of NUM_FRAMES
            for i in range(0, num_images, NUM_FRAMES):
                sequence_files = image_files[i:i+NUM_FRAMES]
                frames = [load_frame(fp) for fp in sequence_files]
                # If the chunk has fewer than NUM_FRAMES, pad with zeros
                if len(frames) < NUM_FRAMES:
                    pad_count = NUM_FRAMES - len(frames)
                    pad_frame = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=np.float32)
                    frames.extend([pad_frame] * pad_count)
                X.append(np.array(frames))
                y.append(current_label)
                print(f"  Loaded sample from {word_folder}: images {i+1} to {i+len(sequence_files)}")
            current_label += 1

    return np.array(X), np.array(y), label_map

In [7]:
print("Loading dataset from Frames_words_level...")
X, y, label_map = load_dataset(DATASET_DIR)
print(f"\nDataset loaded: {X.shape[0]} samples, {len(label_map)} classes.")
print("Label map:", label_map)

if X.shape[0] < 2:
    raise ValueError("Not enough samples loaded. Check the dataset path and structure.")

# Convert labels to one-hot encoding
num_classes = len(label_map)
y_cat = to_categorical(y, num_classes=num_classes)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)
print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

Loading dataset from Frames_words_level...

Processing label: A LOT
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\A LOT: images 1 to 4

Processing label: ABUSE
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\ABUSE: images 1 to 5

Processing label: AFRAID
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\AFRAID: images 1 to 6

Processing label: AGREE
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\AGREE: images 1 to 7

Processing label: ALL
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\ALL: images 1 to 4

Processing label: ANGRY
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\ANGRY: images 1 to 10

Processing label: ANYTHING
  Loaded sample from C:\Users\user\Downloads\ISL_CSLRT_Corpus\Frames_Word_Level\ANYTHING: images 1 to 4

Processing label: APPRECIATE
  Loaded sample from C:\Users\user\Downl

In [8]:
input_shape = (NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, 3)
sequence_input = Input(shape=input_shape)

# Create the CNN base using MobileNetV2
cnn_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
cnn_output = GlobalAveragePooling2D()(cnn_base.output)
cnn_model = Model(inputs=cnn_base.input, outputs=cnn_output)
cnn_model.trainable = False  # Initially freeze the CNN base

# Apply the CNN to each frame using TimeDistributed
encoded_frames = TimeDistributed(cnn_model)(sequence_input)

# Use a Bidirectional LSTM for better temporal feature learning
bi_lstm = Bidirectional(LSTM(64, return_sequences=False))(encoded_frames)
dropout = Dropout(0.5)(bi_lstm)
output = Dense(num_classes, activation='softmax')(dropout)

model = Model(inputs=sequence_input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 10, 224, 224, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 10, 1280)            │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │         688,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 114)                 │          14,706 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,961,330 (11.30 MB)

 Trainable params: 703,346 (2.68 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
print("\nStarting initial training with frozen CNN base...")
history1 = model.fit(X_train, y_train,
                     validation_data=(X_test, y_test),
                     epochs=INITIAL_EPOCHS,
                     batch_size=BATCH_SIZE)




Starting initial training with frozen CNN base...
Epoch 1/50
2/8 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.0469 - loss: 4.8751  

In [ ]:
print("\nUnfreezing the last few layers of MobileNetV2 for fine-tuning...")
# Unfreeze the last 20 layers of MobileNetV2
for layer in cnn_base.layers[-20:]:
    layer.trainable = True

# Compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

print("\nStarting fine-tuning training...")
history2 = model.fit(X_train, y_train,
                     validation_data=(X_test, y_test),
                     epochs=FINETUNE_EPOCHS,
                     batch_size=BATCH_SIZE)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


In [ ]:
idx = 0  # choose a sample index from the test set
sample = X_test[idx]  # shape: (NUM_FRAMES, IMG_HEIGHT, IMG_WIDTH, 3)
true_label = np.argmax(y_test[idx])
pred_probs = model.predict(np.expand_dims(sample, axis=0))
pred_label = np.argmax(pred_probs)

print(f"\nTrue label index: {true_label}, Predicted label index: {pred_label}")
print("Corresponding word labels:")
for word, idx_val in label_map.items():
    if idx_val == true_label:
        print("  True word:", word)
    if idx_val == pred_label:
        print("  Predicted word:", word)
